In [1]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)

filter_param = catalog.load('params:openaire_fetch_options.filter_param')
filter_value = catalog.load('params:openaire_fetch_options.filter_value')
df = catalog.load('raw/openaire/researchproduct#parquet')

[02/28/25 12:29:31] INFO     Loading data from params:openaire_fetch_options.filter_param       ]8;id=553784;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=74287;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openaire_fetch_options.filter_value       ]8;id=759852;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=837525;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from raw/openaire/researchproduct#parquet             ]8;id=912017;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=682321;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185373 entries, 0 to 185372
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   author              185305 non-null  object
 1   openAccessColor     53613 non-null   object
 2   publiclyFunded      121560 non-null  object
 3   type                185373 non-null  object
 4   language            185373 non-null  object
 5   subjects            183336 non-null  object
 6   mainTitle           185373 non-null  object
 7   description         183327 non-null  object
 8   publicationDate     185361 non-null  object
 9   publisher           76578 non-null   object
 10  source              68904 non-null   object
 11  format              170917 non-null  object
 12  bestAccessRight     99741 non-null   object
 13  container           48356 non-null   object
 14  id                  185373 non-null  object
 15  originalId          185373 non-null  object
 16  in

In [3]:
df.columns


Index(['author', 'openAccessColor', 'publiclyFunded', 'type', 'language',
       'subjects', 'mainTitle', 'description', 'publicationDate', 'publisher',
       'source', 'format', 'bestAccessRight', 'container', 'id', 'originalId',
       'indicators', 'instance', 'isGreen', 'isInDiamondJournal',
       'contributor', 'pid', 'country', 'contactPerson', 'embargoEndDate',
       'coverage', 'version', 'size', 'filter_applied'],
      dtype='object')

## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [ ]:
def land_openaire_researchproduct(filter_param, filter_value, df: pd.DataFrame)-> pd.DataFrame:

    expected_columns = [
        'id',
        'openAccessColor',
        'publiclyFunded',
        'type',
        'language',
        'country',
        'mainTitle',
        'description',
        'publicationDate',
        'format',
        'bestAccessRight',
        'indicators',
        'isGreen',
        'isInDiamondJournal',
        'publisher',
        'source',
        'container',
        'contributor',
        'contactPerson',
        'coverage',
        'contactPerson',
        'embargoEndDate',
    ]

    # Agregar columnas faltantes con NaN
    for col in expected_columns:
        if col not in df.columns:
            df[col] = pd.NA

    df = df.convert_dtypes()

    df_researchproduct = df[expected_columns].copy()
    df.reset_index(drop=True, inplace=True)

    # language
    df_researchproduct['language_code'] = df_researchproduct['language'].apply(lambda x: x['code'])
    df_researchproduct['language_label'] = df_researchproduct['language'].apply(lambda x: x['label'])

    ## bestAccessRight
    df_researchproduct['bestAccessRight_label'] = df['bestAccessRight'].apply(lambda x: x['label'] if x else None)
    df_researchproduct['bestAccessRight_scheme'] = df['bestAccessRight'].apply(lambda x: x['scheme'] if x else None)

    ## indicators
    df_indicators = pd.json_normalize(df['indicators']).reset_index(drop=True)
    
    indicators_expected_columns = [
        "citationImpact.citationClass",
        "citationImpact.citationCount",
        "citationImpact.impulse",
        "citationImpact.impulseClass",
        "citationImpact.influence",
        "citationImpact.influenceClass",
        "citationImpact.popularity",
        "citationImpact.popularityClass",
        "usageCounts.downloads",
        "usageCounts.views",
    ]

    # Agregar columnas para indicators y faltantes con NaN
    for col in indicators_expected_columns:
        if col not in df_indicators.columns:
            df_indicators[col] = pd.NA

    df_researchproduct = pd.concat([df_researchproduct.drop(columns=['indicators']).reset_index(drop=True), df_indicators], axis=1)

    # TODO country
    # TODO description
    # TODO format
    # TODO instance
    # TODO source
    # TODO container
    # TODO contributor
    # TODO contactPerson
    # TODO coverage

    ## drop de columnas procesadas en otros df
    df_researchproduct.drop(columns=[
        'country', 'bestAccessRight', 
        'language', 'format',  
        'container', 'source', 'description',
        'contributor', 'contactPerson', 'coverage'
        ], inplace=True)

    df_researchproduct['load_datetime'] = date.today()

    df_researchproduct[filter_param] = filter_value

    return df_researchproduct


In [10]:
df_researchproduct = land_openaire_researchproduct(filter_param, filter_value, df)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df_researchproduct = land_openaire_researchproduct(filter_param, filter_value, df)           │
│   2                                                                                              │
│                                                                                                  │
│ in land_openaire_researchproduct:80                                                              │
│                                                                                                  │
│   77 │   # TODO coverage                                                                         │
│   78 │                                                                                           │
│   79 │   ## drop de columnas procesadas en otros df                                              │
│ ❱ 80 │   df_researchproduct.drop(columns=[                                                       │
│   81 │   │   'author', 'country', 'subjects','bestAccessRight',                                  │
│   82 │   │   'language', 'format', 'instance', 'originalId',                                     │
│   83 │   │   'container', 'source', 'pid', 'description',                                        │
│                                                                                                  │
│ /home/pablo/.local/lib/python3.10/site-packages/pandas/core/frame.py:5581 in drop                │
│                                                                                                  │
│    5578 │   │   falcon  speed   320.0   250.0                                                    │
│    5579 │   │   │   │   weight  1.0     0.8                                                      │
│    5580 │   │   """                                                                              │
│ ❱  5581 │   │   return super().drop(                                                             │
│    5582 │   │   │   labels=labels,                                                               │
│    5583 │   │   │   axis=axis,                                                                   │
│    5584 │   │   │   index=index,                                                                 │
│                                                                                                  │
│ /home/pablo/.local/lib/python3.10/site-packages/pandas/core/generic.py:4788 in drop              │
│                                                                                                  │
│    4785 │   │                                                                                    │
│    4786 │   │   for axis, labels in axes.items():                                                │
│    4787 │   │   │   if labels is not None:                                                       │
│ ❱  4788 │   │   │   │   obj = obj._drop_axis(labels, axis, level=level, errors=errors)           │
│    4789 │   │                                                                                    │
│    4790 │   │   if inplace:                                                                      │
│    4791 │   │   │   self._update_inplace(obj)                                                    │
│                                                                                                  │
│ /home/pablo/.local/lib/python3.10/site-packages/pandas/core/generic.py:4859 in _drop_axis        │
│                                                                                                  │
│    4856 │   │   │   │   # Check if label doesn't exist along axis                                │
│    4857 │   │   │   │   labels_missing = (axis.get_indexer_

In [ ]:
df_researchproduct

,id,openAccessColor,publiclyFunded,type,language,country,mainTitle,description,publicationDate,format,bestAccessRight,isGreen,isInDiamondJournal,publisher,source,container,contributor,contactPerson,coverage,contactPerson,embargoEndDate,language_code,language_label,bestAccessRight_label,bestAccessRight_scheme,usageCounts,citationImpact.citationClass,citationImpact.citationCount,citationImpact.impulse,citationImpact.impulseClass,citationImpact.influence,citationImpact.influenceClass,citationImpact.popularity,citationImpact.popularityClass,usageCounts.downloads,usageCounts.views,load_datetime,relOrganizationId
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,Variability in the growth rates of Saanen kids...,[This study was carried out to determine the v...,2023-12-18,[application/pdf],"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",False,False,"Universidad Nacional Mayor de San Marcos, Facu...",[Revista de Investigaciones Veterinarias del P...,"{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,<NA>,spa,Spanish; Castilian,OPEN,http://vocabularies.coar-repositories.org/docu...,NaN,C5,0.0,0.0,C5,2.841867e-09,C5,2.885067e-09,C5,NaN,NaN,2025-02-28,openorgs____::40b9f835648a3e0d057d6917dd7e54d5
1,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,<NA>,False,publication,"{'code': 'Español', 'label': 'Español'}",None,Estudio de la viabilidad del algoritmo super-t...,[En este trabajo se presenta el diseño de un c...,2013-01-01,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",True,False,<NA>,[reponame:Digital.CSIC. Repositorio Institucio...,None,"[European Commission, Ministerio de Economía y...",None,None,None,<NA>,Español,Español,OPEN,http://vocabularies.coar-repositories.org/docu...,NaN,C5,0.0,0.0,C5,2.841867e-09,C5,7.596338e-10,C5,NaN,NaN,2025-02-28,openorgs____::40b9f835648a3e0d057d6917dd7e54d5
2,core_ac_uk__::a391293fe8bd6c9c6c203ee15e8c2e8f,<NA>,False,publication,"{'code': 'eng', 'label': 'English'}",None,The Three Hundred project: The gas disruption ...,[We analyse the gas content evolution of infal...,2021-03-01,[<nobr>PDF</nobr>],"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",True,False,Oxford University Press (OUP),"[0035-8711, 1365-2966]",None,None,None,None,None,<NA>,eng,English,OPEN,http://vocabularies.coar-repositories.org/docu...,NaN,C5,0.0,0.0,C5,2.841867e-09,C5,2.155773e-09,C5,NaN,NaN,2025-02-28,openorgs____::40b9f835648a3e0d057d6917dd7e54d5
3,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,<NA>,False,publication,"{'code': 'esl/spa', 'label': 'Spanish'}","[{'code': 'AR', 'label': 'Argentina', 'provena...",Evolución del sistema de gestión de incidentes...,[Esta tesina detalla el desarrollo de un siste...,2023-06-23,[application/pdf],"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",True,False,<NA>,None,None,"[Lanfranco, Einar Felipe, Venosa, Paula]",None,None,None,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,NaN,C5,0.0,0.0,C5,2.841867e-09,C5,2.885067e-09,C5,NaN,NaN,2025-02-28,openorgs____::40b9f835648a3e0d057d6917dd7e54d5
4,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,gold,False,publication,"{'code': 'eng', 'label': 'English'}","[{'code': 'AR', 'label': 'Argentina', 'provena...",Responsabilidad Social Corporativa en Empresas...,[El presente artículo presenta un conjunto de ...,2013-12-01,"[application/pdf, 143-162]","{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",False,False,Universidad Nacional de La Plata,"[Proyecciones, Iss 8 (2014)]","{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,<NA>,eng,English,OPEN,http://vocabularies.coar-repositories.org/docu...,NaN,C5,0.0,0.0,C5,2.841867e-09,C5,7.596338e-10,C5,NaN,NaN,2025-02-28,openorgs____::40b9f835648a3e0d057d6917dd7e54d5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185368,portalunbbra::1a507fabf2

In [ ]:
df_researchproduct2pid

,id,scheme,value,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,NaN,2025-02-21
1,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,handle,10261/97737,2025-02-21
2,core_ac_uk__::a391293fe8bd6c9c6c203ee15e8c2e8f,NaN,NaN,2025-02-21
3,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,NaN,2025-02-21
4,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,NaN,NaN,2025-02-21
...,...,...,...,...
280751,portalunbbra::1a507fabf255873c7efa9a67b3d081a5,NaN,NaN,2025-02-21
280752,portalunbbra::d22e22c3f5f77ddd0f037c0a1da23a9b,NaN,NaN,2025-02-21
280753,portalunbbra::fe22079ac12e4b48e955751af4ce027b,NaN,NaN,2025-02-21
280754,revistasumft::106b3b4f6115356de22e7aaf07960b34,NaN,NaN,2025-02-21


In [ ]:
df_researchproduct2url

,id,url,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,https://revistasinvestigacion.unmsm.edu.pe/ind...,2025-02-21
1,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,http://hdl.handle.net/10261/97737,2025-02-21
2,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,http://hdl.handle.net/10261/97737,2025-02-21
3,core_ac_uk__::a391293fe8bd6c9c6c203ee15e8c2e8f,https://nottingham-repository.worktribe.com/fi...,2025-02-21
4,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,http://sedici.unlp.edu.ar/handle/10915/154923,2025-02-21
...,...,...,...
442406,portalunbbra::1a507fabf255873c7efa9a67b3d081a5,http://periodicos.unb.br/index.php/Musica/arti...,2025-02-21
442407,portalunbbra::d22e22c3f5f77ddd0f037c0a1da23a9b,http://periodicos.unb.br/index.php/revistadoce...,2025-02-21
442408,portalunbbra::fe22079ac12e4b48e955751af4ce027b,http://periodicos.unb.br/index.php/belasinfiei...,2025-02-21
442409,revistasumft::106b3b4f6115356de22e7aaf07960b34,https://periodicoscientificos.ufmt.br/ojs/inde...,2025-02-21
